In [32]:
import carla, time, sys, math
sys.path.insert(0,'C:/svs/CARLA_0.9.15/WindowsNoEditor/PythonAPI/carla')
from agents.navigation.basic_agent import BasicAgent

import numpy as np
from enum import Enum

# setup

In [2]:
client = carla.Client("localhost", 2000)
world = client.get_world()

if not world.get_map().name == 'Carla/Maps/Town03':
    world = client.load_world('Town03')

# Support function

In [3]:
def destroy_all_vehicles(world):
    # Ottieni tutti gli attori nella mappa
    all_actors = world.get_actors()
    
    # Filtra solo i veicoli
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    # Distruggi ciascun veicolo
    for vehicle in vehicles:
        vehicle.destroy()
        print(f"Veicolo distrutto: {vehicle.id}")

In [4]:
def spawn_vehicle(world, location, index, y = 0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = carla.Rotation(yaw = y)
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [5]:
def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.brake = 1.0  # Applica il freno massimo
    vehicle.apply_control(control)

# FCW

In [22]:
class Fcw_state(Enum):
    IDLE = 4
    WARNING = 3
    ACTION = 2
    ESCAPE = 1

In [23]:
min_fcw_state = Fcw_state.IDLE
vehicle_vertical_dimension = 0.75
vehicle_lateral_dimension = 0.9
min_ttc = 0.5
average_reaction_time = 2.5
throttle = 0.7
velocity_threshold = 2.77
asphalt_friction_coefficient = 0.8
asphalt_friction_deceleration = 9.81 * asphalt_friction_coefficient

In [24]:
def spawn_radar(vehicle):
    rad_bp = world.get_blueprint_library().find('sensor.other.radar')
    rad_bp.set_attribute('horizontal_fov', str(125))
    rad_bp.set_attribute('vertical_fov', str(115))
    rad_bp.set_attribute('range', str(285))
    rad_bp.set_attribute('points_per_second', str(2000))
    rad_location = carla.Location(x=2.25, z=0.9)
    rad_rotation = carla.Rotation()
    rad_transform = carla.Transform(rad_location,rad_rotation)
    return world.spawn_actor(rad_bp,rad_transform,attach_to=vehicle, attachment_type=carla.AttachmentType.Rigid)

In [25]:
# Convertitori
def check_lateral_collision(azimuth, depth):
    global vehicle_lateral_dimension
    return abs(depth * math.sin(azimuth)) < vehicle_lateral_dimension

def check_vertical_collision(altitude, depth):
    global vehicle_vertical_dimension
    return abs(depth * math.sin(altitude)) < vehicle_vertical_dimension 

def get_projected_velocity(azimuth, altitude, velocity):
    return abs(math.cos(azimuth) * math.cos(altitude) * velocity)

def get_projected_depth(azimuth, altitude, depth):
    return abs(math.cos(azimuth) * math.cos(altitude) * depth)

def get_breaking_distance(projected_velocity):
    global asphalt_friction_deceleration
    return (1/2) * projected_velocity**2 / asphalt_friction_deceleration

def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

In [26]:

def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)



In [27]:
# Simulatore sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation
    weather.precipitation_deposits
    weather.wetness
    weather.dust_storm
    return 0.8

In [82]:
def forward_collision_callback(radar_data, vehicle):
  global min_fcw_state, min_ttc, velocity_threshold, average_reaction_time, asphalt_friction_coefficient, asphalt_friction_deceleration
  asphalt_friction_coefficient = get_asphalt_friction_coefficient()
  asphalt_friction_deceleration = 9.81 * asphalt_friction_coefficient
  for detection in radar_data:
    vehicle_velocity = get_vehicle_velocity(vehicle)
    if vehicle_velocity > velocity_threshold:
      if detection.velocity < 0 and check_lateral_collision(detection.azimuth, detection.depth) and check_vertical_collision(detection.altitude, detection.depth):
        projected_velocity = get_projected_velocity(detection.azimuth, detection.altitude, detection.velocity)   
        breaking_distance = (projected_velocity)
        projected_depth = get_projected_depth(detection.azimuth, detection.altitude, detection.depth)
        reacting_distance = max(0, projected_depth - breaking_distance)
        ttc = reacting_distance / projected_velocity
        fcw_state = Fcw_state.IDLE
        if ttc < min_ttc:
          if vehicle_velocity < projected_velocity - 2*velocity_threshold:
            fcw_state = Fcw_state.ESCAPE
          else:
            fcw_state = Fcw_state.ACTION
        elif ttc < min_ttc + average_reaction_time:
          fcw_state = Fcw_state.WARNING
        if fcw_state.value < min_fcw_state.value:
          min_fcw_state = fcw_state
    else:
      fcw_state = Fcw_state.IDLE

# Setup vehicles

## Vehicle on a slope

In [79]:
obstacle_vehicle = spawn_vehicle(world, carla.Location(x=152.5, y=-33.249748, z=4), 0, -90) 
stop_vehicle(obstacle_vehicle)

## Close vehicle on a slope

In [39]:
 # x=151.715500, y=-19.964308

obstacle_vehicle = spawn_vehicle(world, carla.Location(x=151.715500, y=-19.964308, z=3), 0, -90) 
stop_vehicle(obstacle_vehicle)

## Far away vehicle on a slope

In [71]:
obstacle_vehicle = spawn_vehicle(world, carla.Location(x=153.095108, y=-58.077873, z=7), 0, -90) 
stop_vehicle(obstacle_vehicle)

## Forward vehicle

In [94]:
obstacle_vehicle = spawn_vehicle(world, carla.Location(x=156.463959, y=-5.910003, z=1), 0, 180) 
stop_vehicle(obstacle_vehicle)

In [56]:
obstacle_vehicle.destroy()

True

# Run

In [97]:
vehicle = spawn_vehicle(world, carla.Location(x=178.759705, y=-5.8, z=1), 2, 180)

radar = spawn_radar(vehicle)
radar.listen(lambda radar_data: forward_collision_callback(radar_data, vehicle))

In [96]:

actor_agent = BasicAgent(vehicle)

actor_agent.set_destination(carla.Location(x=153.095108, y=-58.077873, z=7))
actor_agent.set_target_speed(60)
actor_agent.ignore_vehicles()

min_fcw_state = Fcw_state.IDLE

try:
    while True:
        if min_fcw_state != Fcw_state.ACTION:
            vehicle.apply_control(actor_agent.run_step())
        else:
            stop_vehicle(vehicle)
            break
        world.wait_for_tick()
except KeyboardInterrupt:
    print("end")

In [46]:
#destroy_all_vehicles(world)

Numero di veicoli trovati: 2
Veicolo distrutto: 520
Veicolo distrutto: 518


In [98]:
obstacle_vehicle.destroy()
radar.destroy()
vehicle.destroy()

True